In [ ]:
import numpy as np
import time as Time
from numba import jit
import matplotlib.pyplot as plt
import pyvista as pv
import torch

In [ ]:
maskUp = 2.09
maskBottom = 2.6
maskDeep = 2.808

diameter = 60
maskUp_sim = maskUp/maskBottom*diameter
maskDeep_sim = maskDeep/maskBottom*diameter
print(maskUp_sim)
print(maskDeep_sim)

bottom = 100
height = bottom + int(maskDeep_sim)


# vertical mask
film = np.zeros((100, 100, 160, 3))

bottom = 100
height = 140

density = 10

sphere = np.ones((100, 100, 160), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
maskUp = 2.09
maskBottom = 2.6
maskDeep = 2.808

diameter = 260
maskUp_sim = maskUp/maskBottom*diameter
maskDeep_sim = maskDeep/maskBottom*diameter
print(maskUp_sim)
print(maskDeep_sim)

film = np.zeros((300, 300, 400, 3))

bottom = 100
height = bottom + int(maskDeep_sim)

density = 10

sphere = np.ones((300, 300, 400), dtype=bool)

# diameter = 120

center = 150
for k in range(int(diameter/2 - int(maskUp_sim/2))):
    # print(diameter/2 - k)
    radius = diameter/2 - k
    # print('deep', int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*k))
    bottom_step = int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*k)
    for i in range(sphere.shape[0]):
        for j in range(sphere.shape[1]):
            if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
                sphere[i, j, bottom_step:bottom_step+int(maskDeep_sim/(diameter/2 - maskUp_sim/2))] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
depo1 = torch.Tensor(film[:, :center, :, 1]>0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :center, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(film[:, :center, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(film, axis=-1)

print(sumFilm.shape)

In [ ]:
# film = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(film[:, :, :, 5]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etchfilm = film

In [ ]:
import Multi_Species_SF_etching

In [ ]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_SF_etching.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                                         depoThick=120, center_with_direction=np.array([[etchfilm.shape[0]/2,center,75]]), 
                                         range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]]), InOrOut=[1], yield_hist=np.array([None]),
                                        reaction_type=False, param = [1.6, -0.7], N = 300000, 
                                        sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                                        celllength=1e-5, kdtreeN=5, tstep=1e-5,
                                        substrateTop=40,posGeneratorType='top', logname=logname)


# etching = testEtch.depo_position_increase_cosVel_normal(125, int(7e6), 2e-3, 5)


In [ ]:
react_table = np.array([[[0.1, -1, 0, 0], [1.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[0.8, -1, 1, 0], [0.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[1.0,  0, 0, 0], [1.0, 0, -2, 0], [1.0, 0, 0, 0]]])

In [ ]:
Multi_Species_SF_etching.react_table = react_table

In [ ]:
Multi_Species_SF_etching.react_table

In [ ]:
T = 300
Cm = (2*1.380649e-23*T/(27*1.66e-27) )**0.5 # (2kT/m)**0.5 27 for the Al

def max_velocity_u( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.cos(2*np.pi*random2))

def max_velocity_w( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.sin(2*np.pi*random2))

def max_velocity_v( random3):
    return -Cm*np.sqrt(-np.log(random3))

In [ ]:
def velGenerator_benchmark_normal(IN):
    velosity_matrix = np.zeros((IN, 3))
    velosity_matrix[:, 0] = np.random.randn(IN)*0.01 - 0.005
    velosity_matrix[:, 1] = -np.sqrt(2)/2
    velosity_matrix[:, 2] = -np.sqrt(2)/2
    energy = np.linalg.norm(velosity_matrix, axis=1)
    velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
    velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
    velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)
    return velosity_matrix

In [ ]:
# F+ benchmark

N = int(1e7)

velosity_matrix = velGenerator_benchmark_normal(N)

typeID = np.zeros(N)

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
# O2

N = int(1e7)
velosity_matrix = np.zeros((N, 3))
# tstep=1e-5
# celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
Random1 = np.random.rand(N)
Random2 = np.random.rand(N)
Random3 = np.random.rand(N)
velosity_matrix = np.array([max_velocity_u(Random1, Random2), \
                            max_velocity_w(Random1, Random2), \
                                max_velocity_v(Random3)]).T

energy = np.linalg.norm(velosity_matrix, axis=1)
velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)

# velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

# ion_ration = int(N/8)
# typeID[-ion_ration:] = 2
# velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
# F+ Ar+ 
N = int(1e7)
velosity_matrix = np.zeros((N, 3))

typeID = np.zeros(N)

ion_ration = int(N/8)
typeID[-ion_ration:] = 2
velosity_matrix[:, 0] = np.random.rand(N)*0.001
velosity_matrix[:, 1] = np.random.rand(N)*0.001
velosity_matrix[:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
etching = testEtch.inputParticle(125,vel_type_shuffle,int(1e4),int(1e5),int(4e6),-1, 5)

In [ ]:
etching = testEtch.inputParticle(125,vel_type_shuffle,int(1e3),int(4e6),int(1e6),-1, 5)

In [ ]:
# step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(4e4), int(4e6), int(12e5),2, 10, 100)
etching = testEtch.inputParticle(125,vel_type_shuffle,int(1e4),int(5e5),int(1e6),-1, 5)

In [ ]:
testEtch.indice_inject

In [ ]:
testEtch.indice_inject.shape

In [ ]:
testEtch.planes.shape

In [ ]:
np.save('./benchmark_etching/trench_method2_etch_rate_01_12e5', etchfilm)

In [ ]:
etchfilm.shape

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(etchfilm[:, :center, :,0]==density).to_sparse()
# substrute = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]!=density, etchfilm[:, :, :,0]!=0)).to_sparse()
substrute = substrute.indices().numpy().T

# cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>=9, film[:, :, :]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:, :center, :,2]==density).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]!=density, etchfilm[:, :, :,0]!=0)).to_sparse()
substrute = substrute.indices().numpy().T

# cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>=9, film[:, :, :]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:, :, :,0]==density).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etchfilm, axis=-1)

depo1 = torch.Tensor(np.logical_and(etchfilm[:, :, :,1]!=0, etchfilm[:, :, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etchfilm[:, :, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etchfilm[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
# p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
0.6**4

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(etching[0][:25, :, :,1]!=0, etching[0][:25, :, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etching[0][:25, :, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etching[0][:25, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def removeFloat(film):  # fast scanZ
    
    # 获取当前平面的非零元素布尔索引
    current_plane = film != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(film, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= film[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= film[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= film[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= film[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= film[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= film[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition] = 0
    
    return film


In [ ]:
testRemove = removeFloat(sumFilm)

In [ ]:
depo1 = torch.Tensor(testRemove != 0).to_sparse()
depo1 = depo1.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(etching[0][:, :25, :,1]!=0, etching[0][:, :25, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etching[0][:, :25, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etching[0][:, :25, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)
# np.logical_and(etching[0][:, :, :, 5]>0, sumFilm[:, :, :,]>0)
depo1 = torch.Tensor(np.logical_and(etching[0][:, :, :, 5]>0, sumFilm[:, :, :,]>0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==5).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(etching[0][:, :, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==5).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
pointcloud = testEtch.get_pointcloud(sumFilm)

In [ ]:
point_cloud = pv.PolyData(pointcloud[:, 3:])
vectors = pointcloud[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=5,
)

# Display the arrowscyan

# sphere = pv.Sphere(radius=4, center=(21, 75, 114.5))
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
depo1 = torch.Tensor(pointcloud[:,3:]).to_sparse()
depo1 = depo1.indices().numpy().T


depomesh = pv.PolyData(pointcloud[:,3:])
depomesh["radius"] = np.ones(pointcloud[:,3:].shape[0])*0.5
geom = pv.Box()


# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, 50:, :,]!=10, sumFilm[:, 50:, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, 50:, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(testEtch.film, axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
testEtch.parcel.shape

In [ ]:
testEtch.parcel[106]

In [ ]:
testEtch.parcel[100, :3] += testEtch.parcel[100, 3:6] 
print(testEtch.parcel[100])

In [ ]:
testEtch.parcel[10000:11000][:,8].astype(int)
